In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
protein_one = pd.read_csv('/Users/jonathanfarro/Desktop/animal-protein-consumption.csv')

life_ex = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/life-expectancy-hmd-unwpp.csv')

usa_pricing = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/average_prices_for_food.csv')

daily_cals = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/daily-caloric-supply-derived-from-carbohydrates-protein-and-fat.csv')

fruit_per_cap = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/fruit-consumption-per-capita.csv')

veg_per_cap = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/vegetable-consumption-per-capita/vegetable-consumption-per-capita.csv' )

nutrition =  pd.read_excel('/Users/jonathanfarro/Desktop/ Capstone Files/food_comp.xlsx')

protein_per_GDP = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/daily-per-capita-protein-supply-vs-gdp-per-capita.csv')

death = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/death-rate-by-cause-who-mdb.csv')



In [ ]:
# rename columns to something easier to work with 

protein = protein_one.rename(columns={
                        protein_one.columns[3]:'fish',
                        protein_one.columns[4]:'poultry',
                        protein_one.columns[5]:'pork',
                        protein_one.columns[6]:'beef',
                        protein_one.columns[7]:'sheep/goat',
                        protein_one.columns[8]:'other',
                        protein_one.columns[9]:'eggs',
                        protein_one.columns[10]:'milk'
                       })

In [ ]:
# bulk clean up 
protein = protein.rename(columns=lambda x: x.lower())
protein = protein.map(lambda x: x.lower() if isinstance(x, str) else x).reset_index(drop=True)

life_ex = life_ex.rename(columns = lambda x: x.lower())
life_ex = life_ex.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)
life_ex = life_ex.rename(columns = {life_ex.columns[3] : 'life_expectancy(0)'})

vitamins = vitamins.rename(columns = lambda x: x.lower())
vitamins = vitamins.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)

protein_per_GDP = protein_per_GDP.rename(columns = lambda x: x.lower())
protein_per_GDP = protein_per_GDP.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)
gdp_population = protein_per_GDP.drop(columns= {protein_per_GDP.columns[3],protein_per_GDP.columns[6]})
gdp_population = gdp_population.rename(columns = { gdp_population.columns[3]: 'gdp_per_capita'}) 


daily_cals = daily_cals.rename(columns = lambda x: x.lower())
daily_cals = daily_cals.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)

death = death.rename(columns = lambda x: x.lower())
death = death.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)

nutrition = nutrition.rename(columns = lambda x: x.lower())
nutrition = nutrition.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)

fruit_per_cap = fruit_per_cap.rename(columns=lambda x: x.lower())
fruit_per_cap = fruit_per_cap.map(lambda x: x.lower() if isinstance(x, str) else x).reset_index(drop=True)
fruit = fruit_per_cap.rename(columns = {fruit_per_cap.columns[3]:'fruit_kilo_yearly'}).reset_index(drop = True)


In [ ]:
# creating a new column to show total_protein intake for a day.
protein['total_protein'] = protein.iloc[:, 3:11].sum(axis = 1).round(2)
protein.head()

In [ ]:
fruit_per_cap = fruit_per_cap.rename(columns=lambda x: x.lower())
fruit_per_cap = fruit_per_cap.map(lambda x: x.lower() if isinstance(x, str) else x).reset_index(drop=True)
fruit = fruit_per_cap.rename(columns = {fruit_per_cap.columns[3]:'fruit_kilo_yearly'}).reset_index(drop = True)

In [ ]:
capstone = pd.merge(protein,
         life_ex,
         on= ['entity','code','year'],
         how= 'inner')

capstone.head()

In [ ]:
capstone_2 = pd.merge( capstone, gdp_population, on = ['entity','code','year'],how= 'inner')

main_frame = pd.merge(capstone_2,death,  on = ['entity','code','year'],how= 'outer')

final_frame = pd.merge(main_frame, daily_cals,  on = ['entity','code','year'],how= 'outer')

final_frame['total_daily_protein']=final_frame.iloc[:, 3:10].sum(axis=1)

final_frame = final_frame.rename(columns= { 'daily calorie supply per person that comes from animal protein':'daily_cal_animal_protein',
                              'daily calorie supply per person that comes from vegetal protein':'daily_cal_veg_protein',
                              'daily calorie supply per person from fat':'daily_cal_fat',
                              'daily calorie supply per person from carbohydrates':'daily_cal_carbs'})
final_frame

In [ ]:
#creating an all_cause_mortality column, and more cleaning

In [ ]:
final_frame['all_cause_per_100k'] = final_frame.iloc[:,15:24].sum(axis=1)

final_frame_two = final_frame.drop(columns=final_frame.columns[15:24])

final_frame_three = final_frame_two[~final_frame_two['entity'].str.contains(r'\(.*\)')]

final = final_frame_three
final = final.rename(columns={'life_expectancy(0)':'life_exp','population (historical)':'population'})

# what countries have eaten the MOST protein rich diets over these years?

In [ ]:
top_five_total_protein = final.groupby(['entity']) \
                                   .agg({'total_daily_protein': 'sum'}) \
                                   .sort_values(by='total_daily_protein', ascending=False) \
                                   .head(5) \
                                   .reset_index()                      

   
sns.barplot(top_five_total_protein,x = 'entity',y='total_daily_protein',hue='entity');

# what countries have eaten the LEAST protein rich diets over these years?

In [ ]:
final_2 = final.loc[final['total_daily_protein'] != 0.00]

bottom_five_total_protein = final_2.groupby(['entity']) \
                                   .agg({'total_daily_protein': 'sum'}) \
                                   .sort_values(by='total_daily_protein', ascending=True) \
                                   .head(5) \
                                   .reset_index()                      

   
sns.barplot(bottom_five_total_protein,x = 'entity',y='total_daily_protein',hue='entity')
plt.xticks(rotation = 25);

## top 5 countries who derive protein mainly from animal sources.

In [ ]:
animal_protein = final.groupby(['entity']).agg({
    'daily_cal_animal_protein':'sum'}).sort_values(by='daily_cal_animal_protein',ascending = False).head(5).reset_index()
sns.barplot(animal_protein,x = 'entity',y='daily_cal_animal_protein',hue='entity');

# top 5 countries who derive protein from mainly veg 

In [ ]:
veg_pro = final.groupby(['entity']).agg({
    'daily_cal_veg_protein':'sum'}).sort_values(by='daily_cal_veg_protein',ascending = False).head(5).reset_index()
sns.barplot(veg_pro,x = 'entity',y='daily_cal_veg_protein',hue='entity');



In [ ]:
veg_heavy_list = veg_pro.entity.to_list()
animal_heavy_list = animal_protein.entity.to_list()
animal_heavy_list

# how does protein type effect all cause mortality. 

In [ ]:
all_cause_average = final.groupby(['entity']) \
                                   .agg({'all_cause_per_100k': 'sum'}) \
                                   .reset_index()   

animal_all_cause = all_cause_average.loc[all_cause_average['entity'].isin(animal_heavy_list)].sort_values(by='all_cause_per_100k',ascending = False)

veg_all_cause = all_cause_average.loc[all_cause_average['entity'].isin(veg_heavy_list)].sort_values(by='all_cause_per_100k',ascending = False)
animal_all_cause

In [ ]:
sns.barplot(animal_all_cause,x='entity',y='all_cause_per_100k')
sns.barplot(veg_all_cause,x='entity',y='all_cause_per_100k')
plt.xticks(rotation=40);

# how does life expectancy compare beteween top and bottom veg/animal protein? 

In [ ]:
median_life_exp = final.groupby(['entity']) \
                                   .agg({'all_cause_per_100k': 'median'}) \
                                   .reset_index() 

In [ ]:
average_life_exp_animal_protein = median_life.loc[median_life['entity'].isin(animal_heavy_list)]

In [ ]:
average_life_exp_veg_protein = median_life.loc[median_life['entity'].isin(veg_heavy_list)]

In [ ]:
sns.barplot(average_life_exp_animal_protein,x='entity',y='life_exp')
sns.barplot(average_life_exp_veg_protein,x='entity',y='life_exp')
plt.xticks(rotation=40);


In [ ]:
final.groupby(['entity']) \
                                   .agg({'all_cause_per_100k': 'max'}) \
                                   .sort_values(by='all_cause_per_100k', ascending=False) \
                                   .head(5) \
                                   .reset_index()  

# correlation

In [ ]:
correlations = final.iloc[:,2:21].corr()
correlations

In [ ]:
nutrition.iloc[:,2:20].corr()

In [ ]:
final = final.rename(columns={'life_expectancy(0)':'life_exp','population (historical)':'population'})

# nice

In [ ]:
final.columns